
### 1) Use whisper medium model for audio --> text
### 2) Use chat with llama3.2 1B model for text --> summary format

## Audio --> Text


In [ ]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate httpx==0.27.2

In [ ]:
import os
import requests
from IPython.display import Markdown, display, update_display
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer, TextIteratorStreamer
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
AUDIO_MODEL = "openai/whisper-medium"
speech_model = AutoModelForSpeechSeq2Seq.from_pretrained(AUDIO_MODEL, torch_dtype=torch.float16, low_cpu_mem_usage=True, use_safetensors=True)
speech_model.to('cuda')
processor = AutoProcessor.from_pretrained(AUDIO_MODEL)

pipe = pipeline(
    "automatic-speech-recognition",
    model=speech_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    device='cuda',
    return_timestamps=True #important if audio is more than 30sec
)

In [ ]:
result = pipe("/content/sample_data/denver_extract.mp3")

In [ ]:
transcription = result["text"]
print(transcription)

## Text --> Summary Format

In [ ]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
LLAMA = "meta-llama/Llama-3.2-1B-Instruct"

In [ ]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)


In [ ]:
response = tokenizer.decode(outputs[0])

In [ ]:
print(outputs)

## **For Markdown display**

In [ ]:
from IPython.display import Markdown, display, update_display

In [ ]:
display(Markdown(response))